Importing the necessary packages.

In [1]:
import eikon as ek
ek.set_app_key("your app key here")
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pypfopt.cla import CLA
from pypfopt import risk_models
from pypfopt import expected_returns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

We need to retieve the historical components of the Dow Jones Index (DJI). Eikon's `get_data` function is capable of this in theory, i.e. to retieve the components on `date` we could call `ek.get_data(instruments=".DJI", fields=["TR.IndexConstituentRIC"], parameters={'SDate':date})`. However, this does not work in practive because:
* Nothing is returned for dates more than 5 years ago.
* It takes a lot of time to retrieve the data for all trading days even in just the last 5 years.

So instead, we retieve leavers and joiners data and calculate historical components by combining this with the current constituents.

In [2]:
curr, err = ek.get_data(instruments='.DJI', fields=['TR.IndexConstituentRIC'])

In [3]:
curr.head()

,Instrument,Constituent RIC
0,.DJI,GS.N
1,.DJI,NKE.N
2,.DJI,CSCO.OQ
3,.DJI,JPM.N
4,.DJI,DIS.N


In [4]:
leave, err = ek.get_data(['.DJI'], fields=['TR.IndexJLConstituentRIC','TR.IndexJLConstituentChangeDate'],
                        parameters={'IC':'L', 'SDate':'0D', 'EDate':'-20Y'})
leave = leave.set_index('Date')
leave.index = [pd.to_datetime(dt) for dt in leave.index]

In [5]:
join, err = ek.get_data(['.DJI'], fields=['TR.IndexJLConstituentRIC','TR.IndexJLConstituentChangeDate'],
                        parameters={'IC':'J', 'SDate':'0D', 'EDate':'-20Y'})
join = join.set_index('Date')
join.index = [pd.to_datetime(dt) for dt in join.index]

In [6]:
join.head()

,Instrument,Constituent RIC
2020-08-31 00:00:00+00:00,.DJI,HON.N
2020-08-31 00:00:00+00:00,.DJI,AMGN.OQ
2020-08-31 00:00:00+00:00,.DJI,CRM.N
2019-04-02 00:00:00+00:00,.DJI,DOW.N
2018-06-26 00:00:00+00:00,.DJI,WBA.OQ


In [7]:
tickers = list(set(list(curr['Constituent RIC']) + list(leave['Constituent RIC'])))

Retrieving historical EOD prices for all the tickers. The `get_timeseries` call returns at most 3000 rows of data, so the time period from 2000-01-01 had to be divided into two segments. Because of delistings and IPOs later than the end of the first segment, some tickers do not have any data in one of the segments, in which case Eikon returns an error.

However, there seems to be an issue with the Eikon API as it cannot obtain data in some cases when it should. So instead of the below code (which is also a lot of time to run because Eikon takes ages to process the queries) I exported the data using Eikon's Excel add-in and import that data in the notebook below.

In [ ]:
prices = pd.DataFrame()
for ticker in tickers:
    new1 = pd.DataFrame()
    try:
        new1 = ek.get_timeseries(rics=ticker,fields=['CLOSE'],start_date='2000-01-01',end_date='2009-12-31',
                        interval='daily',corax='adjusted')
    except:
        pass
    new2 = pd.DataFrame()
    try:
        new2 = ek.get_timeseries(rics=ticker,fields=['CLOSE'],start_date='2010-01-01',interval='daily',corax='adjusted')
    except:
        pass
    try:
        new = pd.concat([new1,new2],axis=0)
        new.columns = [ticker]
        prices = pd.concat([prices,new],axis=1,join='outer')
    except:
        print('Data for' ,ticker,' completely missing.')

In [8]:
prices = pd.read_excel('C:\Algo trading\Index optimal portfolio project\DOW prices.xlsx',parse_dates=True,index_col='Date')

Checking if we have all the tickers that features in the DJI in the last 20 years.

In [9]:
sum([ticker not in prices.columns for ticker in tickers])

0

In [10]:
prices

,AXP.N,IBM.N,MO.N,KO.N,CVX.N,WMT.N,NKE.N,TRV.N,C.N,EK.N^A12,...,T.N,CAT.N,INTC.OQ,CRM.N,RTX.N,DOW.N,DD.N^I17,JNJ.N,AIG.N,MSFT.OQ
Date,,,,,,,,,,,,,,,,,,,,,
2000-09-11,54.1075,124.7500,6.8715,26.5000,44.6562,54.3125,4.5781,50.6250,528.3115,62.00,...,45.9375,19.1562,64.6875,NaN,18.5423,NaN,38.47,47.5000,1534.202,34.4063
2000-09-12,54.1622,125.0000,6.8715,26.8437,44.2812,54.3750,4.9219,49.7500,525.9788,61.50,...,45.6250,19.0000,64.9375,NaN,18.5423,NaN,39.07,48.3125,1554.100,34.0625
2000-09-13,53.6698,127.3125,6.4953,26.3750,44.3125,53.8750,4.8281,48.5000,518.3982,61.00,...,45.3750,19.3437,61.2500,NaN,18.5607,NaN,38.59,48.8125,1538.391,34.1250
2000-09-14,53.2321,126.8750,6.3507,25.5000,43.7812,52.4375,4.7500,49.4375,517.8152,62.88,...,44.8750,19.5000,59.6250,NaN,18.8741,NaN,38.65,47.9687,1536.296,32.9063
2000-09-15,51.9191,124.8125,6.2783,25.3437,45.1875,51.8750,4.5156,48.8125,510.2346,62.81,...,44.4375,18.5937,57.5156,NaN,18.8556,NaN,38.06,47.5312,1512.209,32.0938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-07,105.6700,122.3000,43.4900,51.0400,81.9300,142.8300,112.4000,116.5700,52.5200,0.03,...,29.4200,148.1800,50.0800,254.70,61.1700,48.60,83.93,148.5900,29.800,214.2500
2020-09-08,103.6700,121.2100,43.0700,49.8100,78.9700,138.4500,112.7200,113.7400,51.0400,0.03,...,29.5100,148.5200,48.9100,241.27,59.2500,47.93,83.93,147.2600,28.890,202.6600
2020-09-09,103.7800,122.2600,43.7900,50.1900,80.0300,139.8900,114.9000,113.6300,51.4000,0.03,...,29.3700,152.6900,49.6200,250.43,59.5800,49.23,83.93,149.7000,29.150,211.2900


Creating a dictionary of `date:list` pairs, where the list contains the constituents on the given date.

In [11]:
constituents_on_date = {prices.index[-1] : list(curr['Constituent RIC'])}
for date in reversed(pd.date_range(prices.index[0],prices.index[-1])):
    leavers = []
    if date in leave.index:
        if 'XOM.N' in leave['Constituent RIC'].loc[date]:
            print(date)
        leavers = leave['Constituent RIC'].loc[date]
        if type(leavers) == str:
            leavers = [leavers]
        else:
            leavers = list(leavers)
    joiners = []
    if date in join.index:
        joiners = join['Constituent RIC'].loc[date]
        if type(joiners) == str:
            joiners = [joiners]
        else:
            joiners = list(joiners)
    constituents_on_date[date+pd.Timedelta(days=-1)] = list(set(constituents_on_date[date]+leavers)-set(joiners))

Checking if we have any missing data for any ticker while the ticker was a constituent.

In [12]:
summed = 0
for date in prices.index:
    for ticker in prices.columns:
        if (ticker in constituents_on_date[date]) and np.isnan(prices[ticker].loc[date]):
            summed += 1
print(summed)

0


Creating a list with the dates when the composion of the index was changed. The optimised portfolio will be rebalanced on these dates.

In [13]:
rebalancing_dates = sorted([dt.tz_localize(None) for dt in set(join.index)])
allocations = {}
display(rebalancing_dates)

[Timestamp('2004-04-08 00:00:00'),
 Timestamp('2008-02-19 00:00:00'),
 Timestamp('2008-09-22 00:00:00'),
 Timestamp('2009-06-08 00:00:00'),
 Timestamp('2012-09-24 00:00:00'),
 Timestamp('2013-09-23 00:00:00'),
 Timestamp('2015-03-19 00:00:00'),
 Timestamp('2017-09-01 00:00:00'),
 Timestamp('2018-06-26 00:00:00'),
 Timestamp('2019-04-02 00:00:00'),
 Timestamp('2020-08-31 00:00:00')]

Retrieving the time series for the index itself.

In [14]:
DJI_1 = ek.get_timeseries(rics='.DJI',fields=['CLOSE'],start_date='2000-01-01',end_date='2009-12-31',
                          interval='daily',corax='adjusted')
DJI_2 = ek.get_timeseries(rics='.DJI',fields=['CLOSE'],start_date='2010-01-01',end_date='2020-09-11',
                          interval='daily',corax='adjusted')
DJI = pd.concat([DJI_1,DJI_2],axis=0)

The price data for some reason contained some (but not all) non-trading days (where each price was the same as on the previous day). The data for the index only contains trading days. The non-trading days are deleted, since it would erroneously increase the values in our (estimated) covariance matrix.

In [15]:
prices['DJI'] = DJI
non_trading_days = prices[prices['DJI'].isnull()].index
prices = prices.drop(non_trading_days,axis=0)

Performing the portfolio optimisation at every rebalancing date. The expected returns are estimated by the mean return up to the given date and similarly for the covariance matrix. (It is definitely possible to improve on this by, for example, using a recency weighted average or shrinking the covariance matrix.)

The optimiser is constrained so that no ticker my receive more than 10% allocation. After the optimiser estimates the efficient frontier, the allocation corresponding to the maximum Sharpe-ratio portfolio is selected. A portfolio with this allocation is held until the next rebalancing date.

The actual optimisation is done using the CLA algorithm as implemented in [PyPortfolioOpt](https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html). PyPortfolioOpt is the best open-source portfolio optimiser available today. It was created and is maintained by Andrew Robert Martin, a friend of mine at Cambridge. For anyone interested in contributing to a finance-related Python package, I highly recommend checking out PyPortfolioOpt [on GitHub](https://github.com/robertmartin8/PyPortfolioOpt).

In [16]:
for date in rebalancing_dates:
    tickers = constituents_on_date[date]
    df = prices[tickers].loc[date-pd.Timedelta(days=365):date]
    
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    cla = CLA(mu,S,weight_bounds=(0,0.1))
    weights = cla.max_sharpe()
    weights = cla.clean_weights()
    for ticker,weight in weights.items():
        if np.isnan(weight):
            weights[ticker] = 0
    
    allocations[date] = weights

Calculating log returns.

In [17]:
rets = np.log(prices/prices.shift(1))[1:]

Functions to calculate the total return and volatility of a portfolio.

In [18]:
def portfolio_return(start, end, tickers, weights):
    return np.dot(rets[tickers].loc[start:end].sum(), weights)

In [19]:
def portfolio_volatility(start, end, tickers, weights):
    return np.sqrt(np.dot(weights, np.dot(rets[tickers].loc[start:end].cov(), weights))) * np.sqrt((end-start).days)

Calculating daily returns on the strategy and the index for plotting.

In [23]:
strat_daily_returns = pd.Series()
for i in range(len(rets)-1):
    date = rets.index[i]
    try:
        prev_rebalance_date = [dt for dt in rebalancing_dates if dt < date][-1]
    except:
        continue
    tickers = [w[0] for w in allocations[prev_rebalance_date].items()]
    weights = [w[1] for w in allocations[prev_rebalance_date].items()]
    daily_return = portfolio_return(date, rets.index[i+1], tickers, weights)
    strat_daily_returns.loc[date] = daily_return
    
dji_daily_returns = rets['DJI'].loc[strat_daily_returns.index[0]:]

Calculating the information ratio for the whole period.

In [24]:
return_diffs = strat_daily_returns-dji_daily_returns[:-1]
np.mean(return_diffs)/np.std(return_diffs) * np.sqrt(252)

0.5445962805528985

Calculating some key statistics, such as the total and annualised returns and volatility on the strategy and the index, and the information ratio for each holding period.

In [87]:
stats = ['Days in period','Strat return','Strat vol','DJI return','DJI vol','Information ratio']
results = pd.DataFrame(index=rebalancing_dates[:-1],columns=stats)
annualised_results = pd.DataFrame(index=rebalancing_dates[:-1],columns=stats)
for i in range(len(rebalancing_dates)-1):
    start = rebalancing_dates[i]
    end = rebalancing_dates[i+1]
    length = 0
    for i in range((end-start).days):
        if start+pd.Timedelta(days=i) in rets.index:
            length += 1    
    
    results.at[start,'Days in period'] = length/100
    results.at[start,'Strat return'] = np.exp(np.sum(strat_daily_returns[start:end]))-1
    results.at[start,'Strat vol'] = np.std(strat_daily_returns[start:end]) * np.sqrt(length)
    results.at[start,'DJI return'] = np.exp(np.sum(dji_daily_returns[start:end]))-1
    results.at[start,'DJI vol'] = np.std(dji_daily_returns[start:end]) * np.sqrt(length)
    results.at[start,'Information ratio'] = np.mean(return_diffs[start:end])/np.std(return_diffs[start:end])*np.sqrt(length)
    
    annualised_results.at[start,'Days in period'] = length/100
    annualised_results.at[start,'Strat return'] = (results['Strat return'].loc[start] + 1) ** (252/length) - 1
    annualised_results.at[start,'Strat vol'] = results['Strat vol'].loc[start] * np.sqrt(252/length)
    annualised_results.at[start,'DJI return'] = (results['DJI return'].loc[start] + 1) ** (252/length) - 1
    annualised_results.at[start,'DJI vol'] = results['DJI vol'].loc[start] * np.sqrt(252/length)
    annualised_results.at[start,'Information ratio'] = results['Information ratio'].loc[start] * np.sqrt(252/length)

The following table displays the key statistics.

**Note:** the numbers represent percentages (except for `Days in period`).

**Note:** the returns represent absolute (not log) returns.

**Note:** the numbers represent aboluste values, NOT annualised.

In [88]:
display(results*100)

,Days in period,Strat return,Strat vol,DJI return,DJI vol,Information ratio
2004-04-08,971.00,75.84,34.80,18.15,23.55,150.52
2008-02-19,150.00,-12.30,15.63,-10.79,17.88,-10.74
2008-09-22,178.00,-31.49,42.73,-23.04,38.67,-32.82
2009-06-08,832.00,140.85,35.87,54.73,30.65,156.30
2012-09-24,249.00,22.02,15.73,13.42,10.64,57.74
2013-09-23,374.00,70.32,20.47,16.23,13.86,242.07
2015-03-19,620.00,48.85,27.83,21.64,19.52,96.72
2017-09-01,204.00,36.47,20.25,10.64,13.15,133.97
2018-06-26,192.00,23.27,23.81,7.94,14.40,74.49
2019-04-02,357.00,9.26,35.70,8.27,36.97,2.62


The following table displays the key statistics.

**Note:** the numbers represent percentages (except for `Days in period`).

**Note:** the returns represent absolute (not log) returns.

**Note:** the numbers represent annualised values, not the actual values in the time period. For example, between 2004-04-08 and 2008-02-19, the DJI actually increased about 18%, and the 4.42% figure in table shows that this translates into a 4.42% yearly return.

In [89]:
display(annualised_results*100)

,Days in period,Strat return,Strat vol,DJI return,DJI vol,Information ratio
2004-04-08,971.00,15.78,17.73,4.42,11.99,76.68
2008-02-19,150.00,-19.78,20.26,-17.46,23.18,-13.92
2008-09-22,178.00,-41.46,50.84,-30.98,46.01,-39.05
2009-06-08,832.00,30.50,19.74,14.13,16.87,86.02
2012-09-24,249.00,22.31,15.83,13.59,10.71,58.09
2013-09-23,374.00,43.16,16.80,10.67,11.38,198.71
2015-03-19,620.00,17.55,17.74,8.29,12.44,61.66
2017-09-01,204.00,46.82,22.50,13.30,14.61,148.90
2018-06-26,192.00,31.60,27.28,10.55,16.50,85.33
2019-04-02,357.00,6.45,30.00,5.77,31.06,2.20


Calculating the cumulative log return on the strategy in daily resolution. For each holding period, the leverage on the strategy portfolio is set so as to make the volatility of the benchmark and the strategy equal.

In [119]:
strat_cum_returns = pd.Series(strat_daily_returns.iloc[0], index=[strat_daily_returns.index[0]])
for i in range(len(rebalancing_dates)-1):
    start = rebalancing_dates[i]
    end = rebalancing_dates[i+1]
    vol_adjustment = results['DJI vol'].loc[start] / results['Strat vol'].loc[start]
    for j in range(1, len(strat_daily_returns.loc[start:end].index)):
        date = strat_daily_returns.loc[start:end].index[j]
        prev_date = strat_daily_returns.loc[start:end].index[j-1]
        strat_cum_returns[date] = strat_cum_returns.loc[prev_date] + strat_daily_returns.loc[date] * vol_adjustment

Doing the same for the benchmark.

In [120]:
dji_cum_returns = dji_daily_returns.expanding().sum()

Turning the log returns into portfolio values.

In [121]:
strat_real_returns = strat_cum_returns.apply(lambda x: np.exp(x)-1) + 1
dji_real_returns = dji_cum_returns.apply(lambda x: np.exp(x)-1) + 1

Plotting the two cumulative log return series.

In [122]:
fig = make_subplots(shared_xaxes=True)
fig.add_trace(go.Line(x=strat_cum_returns.index, y=strat_cum_returns, name='Strategy'))
fig.add_trace(go.Line(x=dji_cum_returns.index, y=dji_cum_returns, name='DJI'))
fig.update_xaxes(title_text="time")
fig.update_yaxes(title_text="Cumulative log return on portfolio")
fig.show()

Plotting the portfolio values for an initial investment of 1.

In [127]:
fig = make_subplots(shared_xaxes=True)
fig.add_trace(go.Line(x=strat_real_returns.index, y=strat_real_returns, name='Strategy'))
fig.add_trace(go.Line(x=dji_real_returns.index, y=dji_real_returns, name='DJI'))
fig.update_xaxes(title_text="time")
fig.update_yaxes(title_text="Portfolio value")
fig.show()

C:\Programming\Anaconda\lib\site-packages\plotly\graph_objs\_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


